In [1]:
from utils.spark_config import get_spark_session
from configs.paths import ROOT_DIR

spark = get_spark_session()
spark

:: loading settings :: url = jar:file:/home/administrator/Desktop/datascience/github/learning-spark/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/administrator/.ivy2.5.2/cache
The jars for the packages stored in: /home/administrator/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5c791278-dddc-4f71-ad8c-d54f41563a1c;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 147ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from central in [default]
	io.delta#delta-storage;4.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.13.1 from central in [default]
	---------------------------------------------------------------------
	|             

In [3]:
churn = spark.sql("SELECT * FROM spark_catalog.delta.`/home/administrator/Desktop/datascience/github/learning-spark/data/gold/customer_churn_gold`")
churn.show()

25/08/21 11:34:44 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException
25/08/21 11:34:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-----+-------------------------------------+-----------------------------+----------------------------+---------------------------------+---------------------------+---------------------------+----------------------------------------+------------------------------------+---------------------------------------------+-----------------------------------------------+-------------+-------+----------+------+------------+-------------+--------------+------------+----------------+-----------+-----------+---------------+----------------+--------------+------------+
|CustomerID|Churn|InternetService_index_ohe_Fiber_optic|InternetService_index_ohe_DSL|InternetService_index_ohe_No|Contract_index_ohe_Month_to_month|Contract_index_ohe_Two_year|Contract_index_ohe_One_year|PaymentMethod_index_ohe_Electronic_check|PaymentMethod_index_ohe_Mailed_check|PaymentMethod_index_ohe_Credit_card_automatic|PaymentMethod_index_ohe_Bank_transfer_automatic|SeniorCitizen|Partner|Dependents|tenure|PhoneSer

In [6]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

# Create label column from Churn column
indexer = StringIndexer(inputCol="Churn", outputCol="label")

# Create features vector from all numeric columns (excluding CustomerID and Churn)
feature_cols = [col for col in churn.columns if col not in ['CustomerID', 'Churn']]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid="skip")

# Create pipeline to prepare data
pipeline = Pipeline(stages=[indexer, assembler])
pipeline_model = pipeline.fit(churn)
prepared_data = pipeline_model.transform(churn)

# Split the prepared data
train, test = prepared_data.randomSplit([0.8, 0.2], seed=42)

# Train RandomForest model
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100, maxDepth=10, seed=42)
rf_model = rf.fit(train)

25/08/21 11:38:35 WARN DAGScheduler: Broadcasting large task binary with size 1630.7 KiB
25/08/21 11:38:35 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
25/08/21 11:38:36 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/08/21 11:38:37 WARN DAGScheduler: Broadcasting large task binary with size 6.1 MiB


In [7]:
rf_model.transform(test).select("features", "label", "prediction").show(10)

25/08/21 11:38:57 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|(25,[0,3,6,13,14,...|  1.0|       1.0|
|[1.0,0.0,0.0,0.0,...|  0.0|       0.0|
|(25,[1,3,6,10,13,...|  0.0|       0.0|
|(25,[0,4,9,13,14,...|  0.0|       0.0|
|(25,[1,3,6,10,13,...|  1.0|       1.0|
|(25,[2,3,7,13,14,...|  0.0|       0.0|
|(25,[2,4,9,11,12,...|  0.0|       0.0|
|(25,[2,4,6,12,13,...|  0.0|       0.0|
|(25,[1,4,9,11,13,...|  0.0|       0.0|
|(25,[1,5,8,13,14,...|  0.0|       0.0|
+--------------------+-----+----------+
only showing top 10 rows


In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Get predictions
predictions = rf_model.transform(test)

# AUC (area under ROC)
binary_eval = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = binary_eval.evaluate(predictions)
print(f"AUC: {auc}")
# Accuracy
accuracy_eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = accuracy_eval.evaluate(predictions)
print(f"Accuracy: {accuracy}")

# F1 Score
f1_eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = f1_eval.evaluate(predictions)
print(f"F1 Score: {f1}")

25/08/21 11:43:28 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


AUC: 0.8485727029438023


25/08/21 11:43:29 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


Accuracy: 0.8070044709388972


25/08/21 11:43:30 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


F1 Score: 0.7975549250706114


In [13]:
# Save the model
rf_model.write().overwrite().save(f"{ROOT_DIR}/models/churn_model")